In [1]:
import pandas as pd
import sql_functions as sql_int

In [2]:
tournament_result = pd.read_csv('/program/tft_overview/tft_Tables/data_sheets/test_resultsOQ1D1GA.csv')

#### changing points into placement

In [14]:
df_tournament_result = tournament_result.copy()
if 'placement' in df_tournament_result.columns: 
    pass 
else: 
    df_tournament_result['placement'] = df_tournament_result['points'].apply(lambda x: 9-x)
    df_tournament_result = df_tournament_result.drop(columns='points')
           
df_tournament_result[df_tournament_result.duplicated(keep= False, subset=['tournament','group','round','placement']) == True].sort_values(by=['tournament','round','group'])

,playerName,tournament,round,group,placement


#### create Result tables

In [31]:
df_tournament_result = df_tournament_result.drop_duplicates()

df_lobby_result = (pd.pivot_table(df_tournament_result,values='playerName',columns='placement',index=['tournament','round','group'],aggfunc='first')
                   .reset_index()
                   .rename(columns={1:'winner',2:'runnerup',3:'third',4:'fourth',5:'fifth',6:'sixth',7:'seventh',8:'eighth'}))

#### Take tournament ID's

In [22]:
schema = 'tft_tables'
sql_query = f'''select name, tournament_id
                FROM tournaments;
                '''

df_tounament_id = pd.DataFrame(sql_int.get_data_mysql(sql_query),
                                    columns=['tournament','tournament_id'])
df_tounament_push = (pd.merge(df_lobby_result,df_tounament_id, how='left',left_on='tournament',right_on='tournament')
                     .drop(columns='tournament')
                     )

#### Checking tournament result already exists

In [25]:
sql_query = f'''SELECT tournament_id,round,party
                FROM lobbys;
                '''
df_existing_lobby = pd.DataFrame(sql_int.get_data_mysql(sql_query),
                                    columns=['tournament_id','round','group'])
df_no_dups = pd.concat([df_tounament_push,df_existing_lobby],ignore_index=True)
df_no_dups = df_no_dups.drop_duplicates(subset=['tournament_id','group','round'],keep=False).rename(columns={'group': 'party'})

In [65]:
sql_int.push_to_database(df_no_dups,'lobbys',sql_int.get_engine_alchemy(),'tft_tables')

The lobbys table was imported successfully.
